# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data_df=pd.read_csv("../output_data/cities.csv",delimiter=",",index_col=0)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)


In [4]:
locations=data_df[["Lat","Lng"]]
weight=data_df["Humidity"]

In [5]:
fig=gmaps.figure()
humidity_layer=gmaps.heatmap_layer(locations,weights=weight)
fig.add_layer(humidity_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
data_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,kahului,20,US,1603729789,69,20.89,-156.47,23.00,2.89
1,charters towers,99,AU,1603729864,57,-20.10,146.27,25.10,3.18
2,kapaa,41,US,1603729749,73,22.08,-159.32,25.68,1.39
3,cabo san lucas,5,MX,1603729776,48,22.89,-109.91,29.44,1.50
4,vaini,20,TO,1603729736,94,-21.20,-175.20,22.00,3.60


In [7]:
#max Temp <=32, Wind Speed <3, cloudiness<5 
hotel_df=data_df.loc[(data_df["Max Temp"]<=32) & (data_df["Wind Speed"]<=3)&(data_df["Cloudiness"]<=5)]
hotel_df=hotel_df.reset_index(drop=True)


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df["Hotel Name"]=""
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,cabo san lucas,5,MX,1603729776,48,22.89,-109.91,29.44,1.50,
1,saint-philippe,0,RE,1603729786,83,-21.36,55.77,23.00,1.50,
2,chumikan,2,RU,1603729889,87,54.72,135.31,-0.69,2.67,
3,qingdao,1,CN,1603730294,100,36.10,120.37,10.00,1.00,
4,lompoc,1,US,1603729756,82,34.64,-120.46,13.00,2.60,


In [9]:
base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
responses=[]
params={"key":g_key,
       "type":"lodging",
        "radius":5000
       }
for i,row in hotel_df.iterrows():
    params["location"]=f"{row['Lat']},{row['Lng']}"
    response=requests.get(base_url,params=params).json()
    responses.append(response)

In [10]:
for x,row in hotel_df.iterrows():
    try:
        hotel=responses[x]["results"][0]["name"]
        hotel_df.loc[x,"Hotel Name"]=hotel
    except:
        print(f"Missing name for {row['City']}")
    

Missing name for vostok
Missing name for birao
Missing name for rudbar
Missing name for mukhen
Missing name for svetlaya


In [11]:
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,cabo san lucas,5,MX,1603729776,48,22.89,-109.91,29.44,1.50,Hotel Tesoro Los Cabos
1,saint-philippe,0,RE,1603729786,83,-21.36,55.77,23.00,1.50,"Chambres d'hôte ""La Trinité"""
2,chumikan,2,RU,1603729889,87,54.72,135.31,-0.69,2.67,Fortuna
3,qingdao,1,CN,1603730294,100,36.10,120.37,10.00,1.00,Holiday Inn Qingdao City Centre
4,lompoc,1,US,1603729756,82,34.64,-120.46,13.00,2.60,Holiday Inn Express Lompoc


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_hotel = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map

markers=gmaps.marker_layer(locations_hotel,info_box_content=hotel_info)

# Display Map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))